In [1]:
import asyncio
import datetime
import os
import dotenv
import re
import json

import pymongo
import httpx
import bs4
import pandas as pd

In [2]:
def connect_to_db():
    # Load environment variables from .env file
    dotenv.load_dotenv()
    
    # Get MongoDB-URI
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["MDM-Python-MeinProjekt"]

    if "Energie" in db.list_collection_names():
        return db["Energie"]
    else:
        collection = db["Energie"]
        collection.create_index([
            ("country", pymongo.ASCENDING),
            ("datetime", pymongo.ASCENDING),
        ], unique=True)
        return collection

In [3]:
async def scrape_website_data(country, date) -> pd.DataFrame:
    """Access the website with the needed parameters; return a PandasDataFrame"""
    
    # Create List with all 20 Productions-Types
    productiontypes = [
        ("productionType.values", f"B{k:02}") for k in range(1, 21)
    ]
    
        
    async with httpx.AsyncClient() as client:
        result = await client.get(
            url="https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show",
            headers={
                "X-Requested-With": "XMLHttpRequest",
            },
            params=list(
                {
                    "viewType": "GRAPH",
                    "areaType": "CTY",
                    "dateTime.dateTime": f"{date:%d.%m.%Y} 00:00|UTC|DAYTIMERANGE",
                    "dateTime.endDateTime": f"{date:%d.%m.%Y} 00:00|UTC|DAYTIMERANGE",
                    "dateTime.timezone": "UTC",
                    "area.values": f"CTY|{country}!CTY|{country}",
                }.items()) + productiontypes,
            )
            
    # make sure the content is UTF-8 and parse the content with bs4
    assert result.headers["content-type"] == "text/html;charset=UTF-8", result.headers["content-type"]
    soup = bs4.BeautifulSoup(result.content.decode("utf-8"))
    
    # select only the part 'script' and the chart-list of the http-file
    javascript_str = soup.find("script").text
    match = re.search(r"var\s+chart\s*=\s*({.*})\s*;", javascript_str, re.S)
    assert match is not None
    
    # returns the first element of the group
    data = json.loads(match.group(1))
    
    # defines the columns for the dataframe
    columns = {
        k: " ".join(v["title"].split())
        for k, v in
        data["graphDesign"].items()
    }
    
    df = pd.DataFrame(
        data["chartData"]
    ).set_index(data["categoryName"]).astype(float).rename(columns=columns)
    
    # combine time with date to get a real timestamp
    df = df.set_index(pd.MultiIndex.from_arrays(
        [
            [country]*df.shape[0],
            df.index.to_series().apply(
                lambda v: datetime.datetime.combine(date, datetime.time.fromisoformat(v))
            ).dt.tz_localize("UTC"),
        ],
        names=["country", "datetime"],
    ))
    
    return df

In [4]:
def insert_data_to_db(collection, df):
    """Insert the data to the collection; if there is already a data-set with the same location and time,
    an Error is raised, but the rest of the inserts will carry on"""

    data = df.reset_index().to_dict("records")

    collection.insert_many(
        data,
        ordered=False,
    )

In [12]:
async def scraping():
    """Run the program: Scraping the website, Insert it to DB"""

    collection = connect_to_db()
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date - datetime.timedelta(days=7)
    country = "10YCH-SWISSGRIDZ"
    
    date = pd.date_range(start_date, end_date, freq="D")

    collected_dfs = []
    
    for d in date:
        print(f'Working on {d.year}-{d.month}-{d.day}')
        df = await scrape_website_data(country=country, date=d)
        collected_dfs.append(df)
            
    df_to_insert = pd.concat(collected_dfs)  

    print("all data scraped, ready to insert in db")
    insert_data_to_db(collection, df_to_insert)

In [13]:
await scraping()

Working on 2024-3-2
Working on 2024-3-3
Working on 2024-3-4
Working on 2024-3-5
Working on 2024-3-6
Working on 2024-3-7
Working on 2024-3-8
Working on 2024-3-9
all data scraped, ready to insert in db


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: MDM-Python-MeinProjekt.Energie index: country_1_datetime_1 dup key: { country: "10YCH-SWISSGRIDZ", datetime: new Date(1709337600000) }', 'keyPattern': {'country': 1, 'datetime': 1}, 'keyValue': {'country': '10YCH-SWISSGRIDZ', 'datetime': datetime.datetime(2024, 3, 2, 0, 0)}, 'op': {'country': '10YCH-SWISSGRIDZ', 'datetime': Timestamp('2024-03-02 00:00:00+0000', tz='UTC'), 'Wind Onshore Generation': 0.0, 'Solar Generation': 0.0, 'Nuclear Generation': 3020.0, 'Hydro Water Reservoir Generation': 541.0, 'Hydro Run-of-river and poundage Generation': 87.0, 'Hydro Pumped Storage Generation': 133.0, '_id': ObjectId('65ed9406852f5e42c32f37e8')}}, {'index': 1, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: MDM-Python-MeinProjekt.Energie index: country_1_datetime_1 dup key: { country: "10YCH-SWISSGRIDZ", datetime: new Date(170934120